In [ ]:
!pip install pennylane tensorflow==2.14.0

In [ ]:
import pennylane as qml
from pennylane import numpy as np
import tensorflow as tf
tf.get_logger().setLevel('ERROR')

# Parámetros del circuito cuántico
n_qubits = 2
n_layers = 2

# Dispositivo cuántico
dev = qml.device("default.qubit", wires=n_qubits)

# Circuito cuántico
def quantum_circuit(inputs, weights):
    qml.templates.AngleEmbedding(inputs, wires=range(n_qubits))
    qml.templates.StronglyEntanglingLayers(weights, wires=range(n_qubits))
    return [qml.expval(qml.PauliZ(i)) for i in range(n_qubits)]

# Modelo cuántico
@qml.qnode(dev)
def quantum_model(inputs, weights):
    return quantum_circuit(inputs, weights)

# Conversión del modelo para TensorFlow
weights_shape = qml.templates.StronglyEntanglingLayers.shape(n_layers, n_qubits)
quantum_layer = qml.qnn.KerasLayer(quantum_model, weight_shapes={"weights": weights_shape}, output_dim=n_qubits)

# Definición del modelo clásico
classical_model = tf.keras.Sequential([
    tf.keras.layers.Dense(4, activation="relu"),
    tf.keras.layers.Dense(n_qubits, activation="relu"),
    quantum_layer,
    tf.keras.layers.Dense(1, activation="sigmoid")  # Salida binaria
])

# Compilación del modelo
classical_model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

# Generación de datos estructurados
def generate_data(samples):
    X = np.random.random((samples, n_qubits))
    y = (np.sum(X, axis=1) > 1).astype(int)  # Etiqueta: 1 si la suma > 1, 0 en caso contrario
    return X, y

# Crear datos
X_train, y_train = generate_data(100)
X_test, y_test = generate_data(20)

# Entrenamiento
classical_model.fit(X_train, y_train, epochs=10, batch_size=5)

# Evaluación
loss, accuracy = classical_model.evaluate(X_test, y_test)
print(f"Pérdida: {loss}, Precisión: {accuracy}")